In [1]:
%%time
import platform
import boto3
import pandas as pd

version = int(platform.python_version_tuple()[0])
if version < 3:
    from StringIO import StringIO
else:
    from io import StringIO

BUCKET_NAME_GLUE = 'attemp-glue'
BUCKET_NAME_CSVS = 'charbeat-trafic'
path_work = 'gestion.pe/2018/12/'

s3 = boto3.resource('s3')
bucket_csv = s3.Bucket(BUCKET_NAME_CSVS)

list_df = []
col_analysis = 'page_avg_time'
df_up = pd.DataFrame(columns=['dia', col_analysis])

for obj in bucket_csv.objects.filter(Prefix=path_work):
    key = obj.key
    print(key)
    url_up = 'https://s3.amazonaws.com/{}/{}'.format(BUCKET_NAME_CSVS, key)
    df_s3 = pd.read_csv(url_up, parse_dates=['dia'])[['dia', col_analysis]]
    df_s3['mes'] = df_s3['dia'].dt.month 
    
    csv_buffer = StringIO()
    df_s3.to_csv(csv_buffer, index=False)
    s3.Object(BUCKET_NAME_GLUE, key).put(Body=csv_buffer.getvalue(), ACL='public-read')
    # print(help(s3.Object(BUCKET_NAME_GLUE, key)))
    """
    copy_source = {
        'Bucket': BUCKET_NAME_CSVS,
        'Key': key
    }
    s3.meta.client.copy(copy_source, BUCKET_NAME_GLUE, key)
    """

gestion.pe/2018/12/2018-12-18.csv
gestion.pe/2018/12/2018-12-19.csv
gestion.pe/2018/12/2018-12-20.csv
gestion.pe/2018/12/2018-12-21.csv
gestion.pe/2018/12/2018-12-22.csv
gestion.pe/2018/12/2018-12-23.csv
gestion.pe/2018/12/2018-12-24.csv
gestion.pe/2018/12/2018-12-25.csv
gestion.pe/2018/12/2018-12-26.csv
gestion.pe/2018/12/2018-12-27.csv
gestion.pe/2018/12/2018-12-28.csv
gestion.pe/2018/12/2018-12-29.csv
gestion.pe/2018/12/2018-12-30.csv
gestion.pe/2018/12/2018-12-31.csv
CPU times: user 4.4 s, sys: 669 ms, total: 5.07 s
Wall time: 1min 12s


In [2]:
baz = {'a': 1, 'b': 2}
foo, bar = baz
print(foo, bar)

a b


In [3]:
baz = [1, 2]
foo, bar = baz
print(foo, bar)

1 2


In [4]:
baz = 3, 4
foo, bar = baz
print(foo, bar)

3 4


In [3]:
%%time
import dask.dataframe as dd
other = dd.read_csv('s3://alternative-charbeat/*')
other.head()

,tz_time,path,section,page_avg_scroll,page_avg_time,page_scroll_starts,page_total_time,page_uniques,page_views,page_views_loyal,page_views_quality,article_id,dia
0,2019-03-01 17:00,archivo.gestion.pe/economia/cuatro-problemas-q...,economia,1007,28,1,28,1,1,0,1,2176544,2019-03-01
1,2019-03-01 00:00,archivo.gestion.pe/noticia/337942/exportacione...,no section,0,24,0,24,1,1,0,1,13,2019-03-01
2,2019-03-01 16:00,archivo.gestion.pe/noticia/694176/cientificoga...,no section,101,18,1,18,1,1,0,1,10000,2019-03-01
3,2019-03-01 20:00,archivo.gestion.pe/tendencias/normas-internaci...,tendencias,0,26,0,26,1,1,0,1,2080149,2019-03-01
4,2019-03-01 22:00,archivo.gestion.pe/tendencias/publicidad-engan...,tendencias,700,52,1,52,1,1,0,1,2191815,2019-03-01


In [1]:
%%time
## VERSION WITH DASK
import dask.dataframe as dd
import pandas as pd

bucket_name = "aatempjson2"
hour = 19
path_json_spark = 's3://{}/{}/{}/{}/{}/*'.format(
    bucket_name, '2019', '04', '08', hour
)
df = dd.read_csv(
    path_json_spark, sep='@@@', engine='python', header=None, squeeze=False
).compute()

CPU times: user 1.85 s, sys: 475 ms, total: 2.32 s
Wall time: 8.73 s


In [3]:
df.head()

,0
0,"{""project"":""ojo_dev"",""event"":""ctr"",""date"":""201..."


In [4]:
def get_memory_usage(data, deep=True):
    return '{} MB'.format(data.memory_usage(deep=deep).sum() / 1024 ** 2)

get_memory_usage(df)

'0.0008935928344726562 MB'

In [5]:
df.shape

(1, 1)

In [6]:
unique_col = df.columns[0]
df.head()

,0
0,"{""project"":""ojo_dev"",""event"":""ctr"",""date"":""201..."


In [7]:
df = pd.DataFrame(data=''.join(list(df[unique_col])).replace('}{', '} {').split(' '))
df.head()

,0
0,"{""project"":""ojo_dev"",""event"":""ctr"",""date"":""201..."
1,"{""project"":""ojo_dev"",""event"":""ctr"",""date"":""201..."
2,"{""project"":""ojo_dev"",""event"":""ctr"",""date"":""201..."
3,"{""project"":""ojo_dev"",""event"":""ctr"",""date"":""201..."
4,"{""project"":""ojo_dev"",""event"":""ctr"",""date"":""201..."


In [8]:
import json
unique_col = df.columns[0]
df[unique_col] = df[unique_col].apply(json.loads)

In [9]:
df.head()

,0
0,"{'project': 'ojo_dev', 'event': 'ctr', 'date':..."
1,"{'project': 'ojo_dev', 'event': 'ctr', 'date':..."
2,"{'project': 'ojo_dev', 'event': 'ctr', 'date':..."
3,"{'project': 'ojo_dev', 'event': 'ctr', 'date':..."
4,"{'project': 'ojo_dev', 'event': 'ctr', 'date':..."


In [10]:
get_memory_usage(df)

'0.0022125244140625 MB'

In [11]:
for col in df[unique_col][0].keys():
    df[col] = df[unique_col].apply(lambda _: _.get(col, ''))

del df[unique_col]

In [12]:
get_memory_usage(df), df.shape

('0.00301361083984375 MB', (10, 5))

In [13]:
df['hour'] = hour
df['date'] = pd.to_datetime(df['date'], format="%Y%m%d")
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df[['project', 'campain', 'url', 'event', 'year', 'month', 'day', 'hour']]
df.head()

,project,campain,url,event,year,month,day,hour
0,ojo_dev,xxxxx,,ctr,2019,4,5,19
1,ojo_dev,xxxxx,,ctr,2019,4,5,19
2,ojo_dev,xxxxx,,ctr,2019,4,5,19
3,ojo_dev,xxxxx,,ctr,2019,4,5,19
4,ojo_dev,xxxxx,,ctr,2019,4,5,19


In [14]:
get_memory_usage(df), df.shape

('0.0026988983154296875 MB', (10, 8))

In [15]:
df = [tuple(_) for _ in df.values]
df

[('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19),
 ('ojo_dev', 'xxxxx', '', 'ctr', 2019, 4, 5, 19)]

In [16]:
from datetime import datetime
datetime.utcnow().hour

21